In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from UliEngineering.EngineerIO import format_value
from si_prefix import si_format
import plecs_helper as helper
%matplotlib
%matplotlib inline

# Imports and setup
from pint import UnitRegistry
from scipy.signal import find_peaks
from scipy.optimize import fsolve

# pandas display using scientific notation
# pd.set_option('display.float_format', lambda x: f'{x:.3e}')

# use pint
units = UnitRegistry()
units.default_format = "~P.2f"
print(f"Testing units: mu_0 = {(1 * units.mu_0).to_compact(units.tesla * units.meter / units.amp) / 4 / np.pi}")

Using matplotlib backend: TkAgg
Testing units: mu_0 = 0.10 m·µT/A


I wound the current-sensing transformers using ferrite beads which I purchased online. The ferrite cores have two semicircular halves. They have a plastic casing which snaps shut around a wire, pressing the cores together to form a continuous loop. Even when the casing is snapped shut, it is possible there is still a air gap between the cores due to manufacturing defects in the casing. Because $\mu \gg \mu_0$, the reluctance $\mathcal{R}$, and thus the flux, is strongly dependent on the size of the air gap. I show that a gap of a few thousandths of an inch is enough to explain my measurements using a typical value for the permeability of a ferrite core $\mu \approx 3000 \mu_0$.

From Ampere's law, we have that $\mathcal{F} = 1 \cdot I_{through} = \oint H ds = \Phi \cdot \mathcal{R}$, where $\mathcal{R} = \mathcal{R_{core}} + \mathcal{R_{gap}} = \frac{2 \pi r}{\mu A} + \frac{l_{gap}}{\mu_0 A}$, so $\Phi = \frac{I_{through}}{\mathcal{R}} = \frac{I_{through}}{(\frac{2 \pi r}{\mu A} + \frac{l_{gap}}{\mu_0 A})}$.

From Faraday's law, we have that $\Delta V = \frac{d}{dt} \bigl[\Phi\bigr]$. Thus, $\Delta V = \frac{d}{dt} \Bigl[ \frac{I_{through}}{\mathcal{R}} \Bigr] = \frac{1}{\mathcal{R}} \cdot \frac{d}{dt}\bigl[I_{through}\bigr]$.

Assuming $I_{through}$ takes the form $I_{through}(t) = I_{DC} + I_0 sin(\omega t)$, then $\Delta V = \frac{1}{\mathcal{R}} \cdot \frac{dI_{through}(t)}{dt} = \frac{1}{\mathcal{R}} \cdot \omega I_0 cos(\omega t)$. This is shifted from the current function by 90 degrees and scales proportionally to the frequency of the measured current.

In [57]:
def parallel(x, y):
    return 1 / (1 / x + 1 / y)

class FerriteCore():
    def __init__(self, inner_diameter, outer_diameter, length, turns, gap_length = 1 * units.thou, mu_r = 3000, I0 = 0.1 * units.amp, freq = 1 * units.megahertz * units.revolution) -> None:
        # all quantities passed in should have pint units attached
        self.inner_diameter = inner_diameter
        self.outer_diameter = outer_diameter
        self.length = length
        self.turns = turns
        self.gap_length = gap_length
        self.I0 = I0
        self.freq = freq

        self.r = (self.inner_diameter + self.outer_diameter) / 2
        self.A = (self.outer_diameter - self.inner_diameter) * self.length

        self.mu = units.mu_0 * mu_r

        self.core_length = 2 * np.pi * self.r
        self.core_reluctance = (self.core_length - self.gap_length) / (self.mu * self.A)
        self.gap_reluctance = self.gap_length / (units.mu_0 * self.A)
        self.reluctance = self.core_reluctance + self.gap_reluctance
        self.gap_reluctance_factor = self.gap_reluctance / self.reluctance

        self.permeance = 1 / self.reluctance
        self.inductance = self.turns ** 2 * self.permeance

        print(f"Using a radius of {self.r} and an air gap length of {self.gap_length}")
        print(f"The air gap contributes to {np.round(self.gap_reluctance_factor.to_base_units() * 100, 1)}% of the total reluctance.")
        print(f"Inductance: {(self.inductance).to_compact(units.henry)}")

        self.dIdt = self.I0 * self.freq
        self.flux = self.I0 / self.reluctance
        self.flux_linkage = self.flux * self.turns
        self.dV = self.flux_linkage * self.freq
        self.H = self.flux / (2 * np.pi * self.r) * self.reluctance
        self.B_core = self.H * self.mu
        self.B_air = self.H * units.mu_0
        # print(f"Your B value is {self.B_core.to_compact(units.tesla)}. The maximum is on the order of 530mT.")
        # print(f"Your air gap B value is {self.B_air.to_compact(units.tesla)}. The maximum is on the order of 530mT.")
        print(f"Expected voltage difference across the transformer terminals: {self.dV.to_compact(units.volt)}")

V0 = 0.5 * units.volt
Rshunt = 100 * units.ohm
I0 = V0 / Rshunt
freq = 400e3 * units.hertz * units.cycle

darkblue_ferrite = FerriteCore(inner_diameter = 0.35 * units.inch, outer_diameter = 0.625 * units.inch, length = 1.102 * units.inch, turns = 60, gap_length = 6.8 * units.thou, I0 = I0, freq = freq, mu_r = 3000)
pink_ferrite = FerriteCore(inner_diameter = 0.5 * units.inch, outer_diameter = 0.8 * units.inch, length = 1.156 * units.inch, turns = 60, gap_length = 11.2 * units.thou, I0 = I0, freq = freq, mu_r = 3000)
lightblue_ferrite = FerriteCore(inner_diameter = 0.35 * units.inch, outer_diameter = 0.625 * units.inch, length = 1.102 * units.inch, turns = 30, gap_length = 8.5 * units.thou, I0 = I0, freq = freq, mu_r = 3000)


Using a radius of 0.49 in and an air gap length of 6.80 th
The air gap contributes to 87.00% of the total reluctance.
Inductance: 4.45 mH
Expected voltage difference across the transformer terminals: 932.79 mV
Using a radius of 0.65 in and an air gap length of 11.20 th
The air gap contributes to 89.20% of the total reluctance.
Inductance: 3.17 mH
Expected voltage difference across the transformer terminals: 664.62 mV
Using a radius of 0.49 in and an air gap length of 8.50 th
The air gap contributes to 89.30% of the total reluctance.
Inductance: 914.63 µH
Expected voltage difference across the transformer terminals: 383.12 mV
